In [72]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [105]:
x = [[39,40,41], [1,2,3], [4,5,6], [3,4,5], [9,10,11], [200,201,202], [123,124,125]]
y = [1,1,1,1,1,1,1]

In [106]:
dim_fake_in = 100
dim_fake_out = 3
dim_real_in = 3
dim_real_out = 1

In [107]:
batch_size = 1 #len(x) 
batch_no = int(len(x)/batch_size)
num_epoch = 1000
learning_rate = 0.001

In [108]:
# Discriminator
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(dim_real_in, 100),
            nn.Sigmoid(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, dim_real_out), 
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.dis(x)
        return x

In [109]:
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(dim_fake_in, 100),
            nn.Sigmoid(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, dim_fake_out)
        )

    def forward(self, x):
        x = self.gen(x)
        return x

In [110]:
D = discriminator()
G = generator()

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)
g_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)

In [111]:
z = Variable(torch.randn(dim_fake_in).float())
print (z)
fake_seq = G(z)
print (fake_seq)

tensor([-0.0529, -0.3957,  1.6574, -2.5071, -0.0315, -0.7634,  2.1508,  0.3674,
         0.0926, -1.4116,  2.5305,  0.8315, -1.8872,  1.9796,  0.0060,  0.8756,
         0.3273,  0.9504,  1.4729,  0.1864,  1.4700, -0.1253, -1.1068,  0.7024,
        -0.2209,  1.4558, -0.9564,  0.1660, -0.4627,  1.1469, -1.1244,  0.1984,
        -0.2962, -0.6126, -1.1484,  0.5382,  0.6646,  0.6597,  0.0182, -0.7468,
         1.1215, -1.3836,  0.6570,  0.0889, -0.2235,  1.2361,  1.0872, -1.1625,
        -0.3709,  0.7862, -0.8034,  0.5779,  0.7875,  1.6914,  0.3653,  0.1121,
        -0.2596,  0.5853, -0.6683,  0.5010,  0.4279, -0.6140,  2.2870, -2.3149,
        -0.2425,  0.1778,  1.5344, -0.1409,  0.5770,  1.2436,  0.5671,  0.3686,
         0.8840, -0.2378,  1.5025, -0.0378,  0.2002, -1.3917,  0.0058,  0.0865,
        -0.1214, -0.0242, -0.1541,  1.3804,  0.6238, -0.6165, -1.2166,  0.1069,
        -0.7375,  0.5667, -0.9235, -0.8794, -1.1791, -0.4146,  0.9183,  1.3234,
        -1.3080,  0.6995,  1.2027, -0.39

In [112]:
z = Variable(torch.tensor(x[0]).float())
print (z)
fake_seq = D(z)
print (fake_seq)

tensor([39., 40., 41.])
tensor([0.4889], grad_fn=<SigmoidBackward>)


In [113]:
# Start training
for epoch in range(num_epoch):
    """
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.tensor(x_train[start:end]).float())
        labels = Variable(torch.tensor(y_train[start:end]).long())
    """
        
    for i in range(len(x)):
        real_seq = Variable(torch.tensor(x[i]).float())
        real_label = Variable(torch.ones(y[i]).float())
        fake_label = Variable(torch.zeros(y[i]).float())
        
        # ===============train discriminator
        
        # compute loss of real_seq
        real_out = D(real_seq)
        d_loss_real = criterion(real_out, real_label)
        real_scores = real_out  # closer to 1 means better

        # compute loss of fake_seq
        z = Variable(torch.randn(dim_fake_in).float())
        fake_seq = G(z)
        fake_out = D(fake_seq)
        d_loss_fake = criterion(fake_out, fake_label)
        fake_scores = fake_out  # closer to 0 means better

        # bp and optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        # ===============train generator
        
        # compute loss of fake_seq
        z = Variable(torch.randn(dim_fake_in).float())
        fake_seq = G(z)
        output = D(fake_seq)
        g_loss = criterion(output, real_label)

        # bp and optimize
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    if epoch % 100 == 0:
            print('Epoch [{}/{}], d_loss: {:.6f}, g_loss: {:.6f} '
              'D real: {:.6f}, D fake: {:.6f}'.format(epoch, num_epoch, d_loss.item(), g_loss.item(),
                  real_scores.data.mean(), fake_scores.data.mean()))

Epoch [0/1000], d_loss: 1.167511, g_loss: 0.696956 D real: 0.624698, D fake: 0.501935
Epoch [100/1000], d_loss: 0.574665, g_loss: 0.893894 D real: 0.982255, D fake: 0.426938
Epoch [200/1000], d_loss: 0.303686, g_loss: 0.936474 D real: 0.999342, D fake: 0.261421
Epoch [300/1000], d_loss: 0.385130, g_loss: 1.182739 D real: 0.999867, D fake: 0.319547
Epoch [400/1000], d_loss: 0.536960, g_loss: 0.953010 D real: 0.999944, D fake: 0.415445
Epoch [500/1000], d_loss: 0.338195, g_loss: 1.102803 D real: 0.999599, D fake: 0.286658
Epoch [600/1000], d_loss: 0.412744, g_loss: 0.860997 D real: 0.995265, D fake: 0.335020
Epoch [700/1000], d_loss: 0.348111, g_loss: 1.197859 D real: 0.997260, D fake: 0.292040
Epoch [800/1000], d_loss: 0.595424, g_loss: 1.271223 D real: 0.997920, D fake: 0.447522
Epoch [900/1000], d_loss: 0.498784, g_loss: 0.730515 D real: 0.997114, D fake: 0.390974


In [114]:
G(Variable(torch.randn(dim_fake_in).float()))

tensor([2.0479, 2.8769, 3.8868], grad_fn=<AddBackward0>)

In [115]:
G(Variable(torch.randn(dim_fake_in).float()))

tensor([1.7534, 2.7657, 3.7440], grad_fn=<AddBackward0>)

In [116]:
D(Variable(torch.tensor([2,113,107]).float()))

tensor([0.0094], grad_fn=<SigmoidBackward>)

In [117]:
D(Variable(torch.tensor([1,2,3]).float()))

tensor([0.4227], grad_fn=<SigmoidBackward>)

In [118]:
D(Variable(torch.tensor([200,201,202]).float()))

tensor([0.9990], grad_fn=<SigmoidBackward>)